# Imports

In [2]:
import requests
import tweepy
import pandas as pd
import math
from datetime import datetime as dt
from datetime import timezone as tz
from alive_progress import alive_bar

# Authentication for Twitter v2 Client

In [3]:
dgorhe_cred = {
    "bearer_token": "AAAAAAAAAAAAAAAAAAAAACC0VQEAAAAAVMb%2BsKdjGQF%2Bk2P0SxmpbqLPFeg%3DGQiBNYXWePAyShoZdbYpUGhLeysplsdxZ8eKMWPfhDUfWpLilo",
    "consumer_key": "BoxdMYUEnBcdl0Rf3dXKCRrKN",
    "consumer_secret": "BosBJ90fJ81EMnznNkgRcncbA6KtrSARwuUkqkdh9EUvgDvvqT",
    "access_token": "1084676618554028034-Me3iM6GE8fkZ0HFtvDnYU7X2kwkxEA",
    "access_token_secret": "JcNVR8vA4guM6T2gQTvQkRNcuJ8nIaPUQxNhaEG9c8ctI",
}

client = tweepy.Client(**dgorhe_cred)

# Reading in List of Twitter Handles

In [4]:
df = pd.read_csv("https://theunitedstates.io/congress-legislators/legislators-current.csv")
congress_twitter_handles = [x for x in list(df["twitter"]) if type(x) == str]

# Extracting Tweets

## Declaring Static Parameters

In [5]:
# Static parameters for tweet search
fields = ['created_at', 'geo', 'lang', 'context_annotations', 'entities']
start_utc = dt.strptime("01/01/2020 00:00:00", "%m/%d/%Y %H:%M:%S")
end_utc = dt.strptime("12/31/2020 11:59:59", "%m/%d/%Y %H:%M:%S")
start_utc_iso8601 = start_utc.astimezone(tz.utc).isoformat('T', timespec='milliseconds').replace("+00:00", "Z")
end_utc_iso8601 = end_utc.astimezone(tz.utc).isoformat('T', timespec='milliseconds').replace("+00:00", "Z")

# Formatted dictionary to 
users_tweets_kwargs = {
    "max_results": 100,
    "start_time": start_utc_iso8601,
    "end_time": end_utc_iso8601,    
    "tweet_fields": fields
}

## Function to Actually Extract Tweets

In [ ]:
def write_to_csv(filename, dictionary):
    with open(filename, "w") as f:
        with alive_bar(len(dictionary.keys()), force_tty=True) as bar:
            for key, value in dictionary.items():
                for val in value:
                    items = [key, val.text, str(val.id), str(val.geo), str(val.lang), str(val.context_annotations), str(val.entities)]
                    line = ",".join(items)
                    f.write(line)
            bar()

In [10]:
# TODO: Add typing
def get_tweets(client, handles, kwargs, dictionary={}):
    count = 0
    checked_handles = []

    # Looping through all twitter handles to get tweets
    with alive_bar(len(handles), force_tty=True) as bar:        
        for handle in handles:
            # If we don't have tweets for handle, get them
            if (handle not in dictionary.keys()):
                congress_person = client.get_user(username=handle)
                kwargs["id"] = congress_person.data.id
                paginator = tweepy.Paginator(client.get_users_tweets, **kwargs)
                dictionary[handle] = []
                
                # Iterate through pages of 100 tweets and append each tweet
                for response in paginator:
                    if (response.data) and (len(response.data) > 0):
                        for tweet in response.data:
                            dictionary[handle].append(tweet)
            bar()

            if (count % 10 == 0 and count != 0):
                write_to_csv("../data/congress_tweets.csv", dictionary)
                checked = ", ".join(list(dictionary.keys()))
                print("Wrote the following handles to csv: ", checked)
                checked_handles.append(list(dictionary.keys()))
                dictionary = {}
                
        
    return dictionary, checked_handles

In [12]:
handles = congress_twitter_handles[:10]
congress_tweets = {}

In [13]:
get_tweets(client, handles, users_tweets_kwargs, congress_tweets)

In [1]:
len(congress_tweets.keys())

NameError: name 'congress_tweets' is not defined

In [41]:
x = json.dumps(congress_tweets["SenSherrodBrown"][0].context_annotations[0])
type(x)

str